#### 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import cvxpy as cp
from scipy import optimize
from scipy.optimize import linprog
from scripts_t.estimation_t import *

In [2]:
rand_seed = 1
num_position = 5
bike_num = 20
lambd = 10
grid_size = 5
beta0 = 1
beta1_true = -1
T = 100
loc_bound = 5

In [3]:
true_pos_ind,bike_num,num_records,book_bike,book_index, \
dist,bike_loc,all_period,num_booked, \
cand_loc,true_loc,position_weight = gen_sync_in_grid(rand_seed,num_position,bike_num,lambd,
                                                     grid_size,beta0=beta0,beta1_true=beta1_true,T=T,loc_bound=loc_bound)
# Print results for verification
print("bike_num:", bike_num)
print("num_records:", num_records)
print("book_bike:", book_bike)
print("book_index:", book_index)
print("dist:", dist)
print("bike_loc:", bike_loc)
print("all_period:", all_period)
print("num_booked:", num_booked)
print("cand_loc:", cand_loc)
print("true_loc:", true_loc)
print("position_weight:", position_weight)

bike_num: 20
num_records: 727
book_bike: [19  7 12  3 14 15  0 10 12  9  8  3  4 19 14 10  8  9 11 12 19  4  5 15
 14  1  0  8  9 10  3 16 12 18  5 11 14  0 18  3  1  9  8 11 17  2  1  5
 16 15  3 14 11  6  5 17  9 16  8 11 14  3  5  1  2 13  6 16  8  2 14  1
 17 14  7  5 16  6 17  8  6 16  2  6  1  8  6  4 11  7 16  4  7  2 15 13
  5  7  1 17 18 15 13  7  5 15 18 17 13  5  2 13 11 18 14  6 18 11 15 17
  5 16 14 11  2 15  5 16  3 14 15  0  2 14 16  3  5  3  1  0 14 19 10 11
  0 14  3 19 17  7  1 10  2  0 19 17 10 14  1  4 14 10 19 17 10  4 15  2
 14  0  7 15  7  0 16  0  1 15  3  7  8 15  0 16 10  1  3  7 19 15 14  0
  3  6 14  3 17 16  6  7 14  3 15 14  7  5 14  6  3 16  6 10  7 14  5 16
 10 15  6 14 19 16  5 10 15 14  4  3  6 14 15 16  3 16  2  4  0  3 19  5
 16  3  5 15  4 19  3  5  2 15 13  4 19  3  2 15  0  2 13  3  1 15  5 14
  4  0 10 15  1  4  5 14  0  3  1  4  0  2  4 13  5 14  2 10  5  1  0 18
 13  8  6 16  5  5  4 18 16  0  8 18  5  6  2 15  8 16  1 13  6 13  4  6
 14 13 16 

#### Implementation of EM algorithm

In [4]:
# Initialize num of position, the true position is given, the number of potential rider arrival position is the same as the whole grid.
num_position_true = num_position
num_position = grid_size**2

w = np.random.uniform(0, 1, num_position)
w = w/np.sum(w)
w = w.reshape(1,-1)

diff_w = np.inf
beta1 = np.repeat(-1, num_position)
beta1 = beta1.reshape(1,-1)
dist_old = caldist(cand_loc[-1],bike_loc,bike_num)
iter = 0
threshold = 1e-4

# EM iteration
while diff_w > threshold:
    iter = iter+1
    p_old = np.zeros((num_position,bike_num+1,num_records))
    p_olds = np.zeros((num_position,num_booked))
    w_old = w[-1,:]
    #Computes the probability that no bike is chosen for each position and record.
    p_old[:,0,:] = 1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2))
    #Computes the probability that each specific bike is chosen for each position
    p_old[:,1:,:] = np.exp(beta0+np.reshape(beta1,(-1,1,1))*np.transpose(dist_old,(0,2,1)))/ \
        np.reshape((1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2)),(num_position,1,-1))
    #Calculates the conditional probabilities for the booked bikes by normalizing over all possible bikes for the observed booking events.
    p_olds = p_old[:,book_bike+1,book_index]*np.reshape(w_old,(-1,1))/np.reshape(np.sum(np.reshape(w_old,(-1,1))*p_old[:,book_bike+1,book_index],axis=0),(1,-1))
    # This is to compute s defined in the paper
    p_oldsj0 = w_old*np.sum(p_old[:,0,:]*all_period,axis=1)/np.sum(w_old*np.sum(p_old[:,0,:]*all_period,axis=1))
    s_old = np.sum((1-np.sum(np.expand_dims(w_old,1)*(1/(1+np.sum(np.exp(beta0+np.reshape(beta1,(-1,1,1))*dist_old),axis=2))),axis=0))*all_period)
    # This is to compute c, we first need N*(T - s)/s, then the summation
    N_oldy = num_booked*(T-s_old)/s_old
    c = np.sum(p_olds,axis=1)+N_oldy*p_oldsj0
    w_star = np.reshape(c/np.sum(c),(1,-1))
    w = np.concatenate((w,w_star),axis=0)
    diff_w = np.sum(np.abs(w[-1,:]-w[-2,:]))

pre_w = w[-2]
true_w = np.zeros(grid_size**2)
true_w[true_pos_ind] = position_weight
sel_ind = np.where(pre_w>=np.sort(pre_w)[-num_position_true])[0]
true_pred_ind = np.intersect1d(sel_ind,true_pos_ind)
l1norm = np.sum(np.abs(pre_w-true_w))
lkd = findlkd_no_constraint(num_position,dist_old,
                                beta0,np.repeat(beta1_true,num_position).reshape(-1,1),pre_w,
                                bike_num,num_records,book_bike,book_index,num_booked,all_period)
print('Number of Iterations:', iter)
print('Accuracy:',true_pred_ind.shape[0]/num_position_true)
print('L-1 norm:',l1norm)
print('Likelihood:', lkd)

Number of Iterations: 387
Accuracy: 0.8
L-1 norm: 0.49687601545433824
Likelihood: -2386.541340799739


#### Implementation of MM algorithm

In [6]:
# Initialization of parameters
num_position = grid_size**2
w = np.random.uniform(0,1,num_position)
w = w/np.sum(w)
w = w.reshape(1,-1)
beta1 = np.repeat(-1,num_position).reshape(-1,1)
loc = cand_loc[-1]

dist1 = caldist(loc,bike_loc,bike_num)
choice_prob = np.zeros((num_position,bike_num+1,num_records))
choice_prob[:,0,:] = 1/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2))
choice_prob[:,1:,:] = np.exp(beta0+beta1.reshape(-1,1,1)*np.transpose(dist1,(0,2,1)))/(1+np.sum(np.exp(beta0+beta1.reshape(-1,1,1)*dist1),axis=2)).reshape(num_position,1,-1)
diff_w = np.inf
threshold = 1e-4
threshold1 = 5e-3
iter = 0
grad_old = 0
diff_grad = np.inf
w_prime = w

# Defines a function for optimizing the line search parameter alpha. It uses the golden-section search method to find the optimal step size alpha that maximizes the objective function along the direction of the vector (ej - w_prime).
def gold_search_func(alpha):
    f = g(w_prime+alpha*(ej-w_prime),choice_prob,book_bike,book_index)-np.sum(grad_h*(w_prime+alpha*(ej-w_prime)))
    return -f

# Frank-Wolf algorithm
while(diff_w>threshold):
    iter += 1
    gw_prime = g(w_prime,choice_prob,book_bike,book_index)
    grad_h = -num_booked *np.sum(choice_prob[:,0,:]*all_period,axis=1)/np.sum((1-np.sum(w.reshape(-1,1)*choice_prob[:,0,:],axis=0))*all_period)
    diff_grad = np.inf
    while (diff_grad>threshold1):
        grad_g = find_grad_g(w_prime,choice_prob,book_bike,book_index)
        j = np.argmax(grad_g-grad_h)
        ej = np.zeros(num_position)
        ej[j] = ej[j]+1
        alpha = optimize.golden(gold_search_func, brack=(0, 1))
        w_prime = w_prime + alpha*(ej-w_prime)
        grad_new = np.abs(gw_prime-np.sum(grad_h*w_prime))
        diff_grad = np.abs(grad_new-grad_old)
        grad_old = grad_new
    diff_w = np.sum(np.abs(w_prime-w))
    w = w_prime

pre_w = w[-1]
true_w = np.zeros(grid_size**2)
true_w[true_pos_ind] = position_weight
sel_ind = np.where(pre_w>=np.sort(pre_w)[-num_position_true])[0]
true_pred_ind = np.intersect1d(sel_ind,true_pos_ind)
l1norm = np.sum(np.abs(pre_w-true_w))
lkd = findlkd_no_constraint(num_position,dist_old,
                                beta0,np.repeat(beta1_true,num_position).reshape(-1,1),pre_w,
                                bike_num,num_records,book_bike,book_index,num_booked,all_period)

print('Number of Iterations:', iter)
print('Accuracy:',true_pred_ind.shape[0]/num_position_true)
print('L1-norm:',l1norm)
print('Likelihood:',lkd)

/home/tagore/repos/python_study/journal_replication/sharebikedemand/scripts_t/estimation_t.py:415: RuntimeWarning: invalid value encountered in log
  g = np.sum(np.log(np.sum(w.reshape(-1,1)*(choice_prob[:,book_bike+1,book_index]),axis=0)))


Number of Iterations: 640
Accuracy: 0.8
L1-norm: 0.48305168890608524
Likelihood: -2386.569354915584
